### Training code for the second version of Roberta I tried, this one has a lot of iterations and uses other groups test sets if you are wondering. The reason the later ones are not finished training is because it was not training for the purpose of saving a model but to verify the F1 score and also i didn't have enough resources to train each model fully and save them. 

In [2]:
!pip install transformers datasets scikit-learn --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12=

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, f1_score
from transformers import EarlyStoppingCallback

# Load your CSV and Excel files from Google Drive
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/group 16_url1.xlsx'

train = pd.read_csv(train_path)
test = pd.read_excel(test_path)

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Load tokenizer
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

# Tokenize all datasets
dataset = dataset.map(tokenize, batched=True)

# Rename label column
dataset["train"] = dataset["train"].rename_column("Emotion", "labels")
#dataset["validation"] = dataset["validation"].rename_column("labels", "labels")
#dataset["test"] = dataset["test"].rename_column("labels", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["Emotion"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
dataset = dataset.cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=12,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Casting the dataset:   0%|          | 0/1233 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.193300,1.795450,0.434712,0.443768
2,0.823000,1.999812,0.446878,0.456150
3,0.592500,2.230114,0.443633,0.452327
4,0.192000,2.785596,0.461476,0.467427
5,0.264000,3.415392,0.464720,0.473903


TrainOutput(global_step=15700, training_loss=0.6765381433995097, metrics={'train_runtime': 1435.0509, 'train_samples_per_second': 87.499, 'train_steps_per_second': 10.94, 'total_flos': 1.6519511409984e+16, 'train_loss': 0.6765381433995097, 'epoch': 5.0})

### These next iterations are using a lexicon that I found which works and does its purpose but I didn't implement it to the model, so right now the model doesnt use the features of the lexicon, an oversight on my part. 

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
import os

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/comb_go.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/group 16_url1.xlsx'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load dataset
train = pd.read_csv(train_path)
test = pd.read_excel(test_path)

# Prepare emotion lexicon with mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)

with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Create emotion count features
def add_lexicon_features(df):
    features = []
    for sentence in df['text']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=sorted(set(emotion_mapping.values())))

# Add features to data
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
for col in train_feats.columns:
    train[f'lex_{col}'] = train_feats[col]
    test[f'lex_{col}'] = test_feats[col]

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

dataset = dataset.map(tokenize, batched=True)

# Rename label column
dataset["train"] = dataset["train"].rename_column("main_category", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["main_category"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
dataset = dataset.cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()

Mounted at /content/drive


<ipython-input-2-6b458f53e0f3>:27: DtypeWarning: Columns (1,2,3,4,5,8,31,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_path)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Casting the dataset:   0%|          | 0/1233 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.067000,1.662603,0.470397,0.481512
2,0.983600,1.805625,0.467153,0.467988
3,0.717900,1.887830,0.503650,0.499909
4,0.460600,2.305369,0.487429,0.499549


TrainOutput(global_step=13972, training_loss=0.818516490301088, metrics={'train_runtime': 7125.3696, 'train_samples_per_second': 62.732, 'train_steps_per_second': 3.922, 'total_flos': 1.0414357722179174e+17, 'train_loss': 0.818516490301088, 'epoch': 4.0})

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_2"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_2/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_2/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_2/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_2/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_2/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_2/tokenizer.json')

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
import os

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups_.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/new_group_.csv'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load dataset
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Prepare emotion lexicon with mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)

with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Create emotion count features
def add_lexicon_features(df):
    features = []
    for sentence in df['Translation']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=sorted(set(emotion_mapping.values())))

# Add features to data
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
for col in train_feats.columns:
    train[f'lex_{col}'] = train_feats[col]
    test[f'lex_{col}'] = test_feats[col]

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["Translation"], padding="max_length", truncation=True, max_length=256)

dataset = dataset.map(tokenize, batched=True)

# Rename label column in all splits
for split in ["train", "validation", "test"]:
    if "Emotion" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("Emotion", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["Emotion"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.930300,0.929808,0.659274,0.663929
2,0.903600,0.889912,0.663306,0.666330
3,0.627100,0.930300,0.705645,0.711544
4,0.501100,1.119775,0.683468,0.689107
5,0.218900,1.500722,0.665323,0.674570


TrainOutput(global_step=8325, training_loss=0.6466675904563239, metrics={'train_runtime': 4278.7198, 'train_samples_per_second': 49.806, 'train_steps_per_second': 3.113, 'total_flos': 6.206301761250816e+16, 'train_loss': 0.6466675904563239, 'epoch': 5.0})

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group9/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group9/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group9/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group9/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group9/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group9/tokenizer.json')

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/goemotions_combined.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/new_group_.csv'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load datasets
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Normalize and clean labels
train["Emotion"] = train["Emotion"].str.strip().str.lower()
test["Emotion"] = test["Emotion"].str.strip().str.lower()

# Filter to only valid labels shared by both train and test
shared_labels = sorted(set(train["Emotion"].unique()) & set(test["Emotion"].unique()))
train = train[train["Emotion"].isin(shared_labels)]
test = test[test["Emotion"].isin(shared_labels)]

# Prepare NRC emotion lexicon mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)
with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Function to add lexicon-based emotion counts
def add_lexicon_features(df):
    features = []
    for sentence in df['Translation']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=[f'lex_{e}' for e in sorted(set(emotion_mapping.values()))])

# Add lexicon features (not directly used in this model version, but useful for later!)
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
train = pd.concat([train.reset_index(drop=True), train_feats], axis=1)
test = pd.concat([test.reset_index(drop=True), test_feats], axis=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["Translation"], padding="max_length", truncation=True, max_length=256)

# Create HuggingFace Datasets
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenize all sets
dataset = dataset.map(tokenize, batched=True)

# Rename label column
for split in ["train", "validation", "test"]:
    if "Emotion" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("Emotion", "labels")

# Create ClassLabel feature
class_names = sorted(shared_labels)
class_label_feature = ClassLabel(names=class_names)

# Cast 'labels' to ClassLabel
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config and load
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train
trainer.train()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/28068 [00:00<?, ? examples/s]

Map:   0%|          | 0/657 [00:00<?, ? examples/s]

Map:   0%|          | 0/657 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/28068 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/657 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/657 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.783900,0.822335,0.799087,0.718125
2,0.778100,0.736137,0.809741,0.743670
3,0.550100,0.873519,0.805175,0.738414
4,0.510200,0.921075,0.799087,0.723140
5,0.477200,1.108052,0.803653,0.733892


TrainOutput(global_step=8775, training_loss=0.6697721025407144, metrics={'train_runtime': 4511.8701, 'train_samples_per_second': 49.767, 'train_steps_per_second': 3.112, 'total_flos': 6.539450756990976e+16, 'train_loss': 0.6697721025407144, 'epoch': 5.0})

In [ ]:
model_path = "/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group(goem)"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group25(goem)/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group25(goem)/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group25(goem)/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group25(goem)/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group25(goem)/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_group25(goem)/tokenizer.json')

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/group 16_url1.xlsx'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load datasets
train = pd.read_csv(train_path)
test = pd.read_excel(test_path)

# Normalize and clean labels
train["Emotion"] = train["Emotion"].str.strip().str.lower()
test["Emotion"] = test["Emotion"].str.strip().str.lower()

# Filter to only valid labels shared by both train and test
shared_labels = sorted(set(train["Emotion"].unique()) & set(test["Emotion"].unique()))
train = train[train["Emotion"].isin(shared_labels)]
test = test[test["Emotion"].isin(shared_labels)]

# Prepare NRC emotion lexicon mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)
with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Function to add lexicon-based emotion counts
def add_lexicon_features(df):
    features = []
    for sentence in df['text']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=[f'lex_{e}' for e in sorted(set(emotion_mapping.values()))])

# Add lexicon features (not directly used in this model version, but useful for later!)
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
train = pd.concat([train.reset_index(drop=True), train_feats], axis=1)
test = pd.concat([test.reset_index(drop=True), test_feats], axis=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

# Create HuggingFace Datasets
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenize all sets
dataset = dataset.map(tokenize, batched=True)

# Rename label column
for split in ["train", "validation", "test"]:
    if "Emotion" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("Emotion", "labels")

# Create ClassLabel feature
class_names = sorted(shared_labels)
class_label_feature = ClassLabel(names=class_names)

# Cast 'labels' to ClassLabel
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config and load
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Compute metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train
trainer.train()


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.200400,1.590209,0.458232,0.458008
2,0.826600,1.929426,0.450933,0.469635
3,0.667200,2.191453,0.442011,0.465789
4,0.450600,2.217572,0.462287,0.473127


TrainOutput(global_step=6280, training_loss=0.8396087808593823, metrics={'train_runtime': 3255.5656, 'train_samples_per_second': 61.711, 'train_steps_per_second': 3.858, 'total_flos': 4.680797541265613e+16, 'train_loss': 0.8396087808593823, 'epoch': 4.0})

In [4]:
model_path = "/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_4"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_4/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_4/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_4/vocab.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_4/merges.txt',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_4/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/best_robertaV2_4/tokenizer.json')

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
import os

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups_.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/new_group_.csv'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load dataset
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Prepare emotion lexicon with mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)

with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Create emotion count features
def add_lexicon_features(df):
    features = []
    for sentence in df['text']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=sorted(set(emotion_mapping.values())))

# Add features to data
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
for col in train_feats.columns:
    train[f'lex_{col}'] = train_feats[col]
    test[f'lex_{col}'] = test_feats[col]

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

dataset = dataset.map(tokenize, batched=True)

# Rename label column in all splits
for split in ["train", "validation", "test"]:
    if "main_category" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("main_category", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["main_category"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/24364 [00:00<?, ? examples/s]

Map:   0%|          | 0/219 [00:00<?, ? examples/s]

Map:   0%|          | 0/220 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/24364 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/220 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/219 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.219200,0.898919,0.677273,0.716814
2,0.698600,0.922440,0.622727,0.673073
3,0.623400,1.071555,0.622727,0.702448
4,0.337500,1.287480,0.618182,0.693838


TrainOutput(global_step=6092, training_loss=0.7512019249810565, metrics={'train_runtime': 3124.236, 'train_samples_per_second': 62.387, 'train_steps_per_second': 3.9, 'total_flos': 4.541191864587878e+16, 'train_loss': 0.7512019249810565, 'epoch': 4.0})

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
import os

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups_.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/new_group_.csv'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load dataset
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Prepare emotion lexicon with mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)

with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Create emotion count features
def add_lexicon_features(df):
    features = []
    for sentence in df['text']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=sorted(set(emotion_mapping.values())))

# Add features to data
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
for col in train_feats.columns:
    train[f'lex_{col}'] = train_feats[col]
    test[f'lex_{col}'] = test_feats[col]

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

dataset = dataset.map(tokenize, batched=True)

# Rename label column in all splits
for split in ["train", "validation", "test"]:
    if "main_category" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("main_category", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["main_category"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/27021 [00:00<?, ? examples/s]

Map:   0%|          | 0/304 [00:00<?, ? examples/s]

Map:   0%|          | 0/305 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/27021 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/305 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/304 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.873100,1.116555,0.613115,0.594053
2,0.804300,1.169155,0.626230,0.603000
3,0.571600,1.115661,0.655738,0.655713
4,0.362400,1.354858,0.659016,0.662986
5,0.284800,1.512751,0.701639,0.691884
6,0.156900,1.928833,0.652459,0.647369


TrainOutput(global_step=10134, training_loss=0.5720374550084426, metrics={'train_runtime': 5198.4474, 'train_samples_per_second': 41.583, 'train_steps_per_second': 2.599, 'total_flos': 7.554642836132966e+16, 'train_loss': 0.5720374550084426, 'epoch': 6.0})

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
import os

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups_.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/new_group_.csv'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load dataset
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Prepare emotion lexicon with mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)

with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Create emotion count features
def add_lexicon_features(df):
    features = []
    for sentence in df['text']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=sorted(set(emotion_mapping.values())))

# Add features to data
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
for col in train_feats.columns:
    train[f'lex_{col}'] = train_feats[col]
    test[f'lex_{col}'] = test_feats[col]

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=256)

dataset = dataset.map(tokenize, batched=True)

# Rename label column in all splits
for split in ["train", "validation", "test"]:
    if "main_category" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("main_category", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["main_category"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.007100,0.998655,0.650633,0.649939
2,0.733500,1.010946,0.655696,0.641025
3,0.532500,1.254175,0.648101,0.636813


KeyboardInterrupt: 

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
import os

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups_.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/new_group_.csv'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load dataset
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Prepare emotion lexicon with mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)

with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Create emotion count features
def add_lexicon_features(df):
    features = []
    for sentence in df['Translation']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=sorted(set(emotion_mapping.values())))

# Add features to data
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
for col in train_feats.columns:
    train[f'lex_{col}'] = train_feats[col]
    test[f'lex_{col}'] = test_feats[col]

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["Translation"], padding="max_length", truncation=True, max_length=256)

dataset = dataset.map(tokenize, batched=True)

# Rename label column in all splits
for split in ["train", "validation", "test"]:
    if "Emotion" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("Emotion", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["Emotion"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/26037 [00:00<?, ? examples/s]

Map:   0%|          | 0/796 [00:00<?, ? examples/s]

Map:   0%|          | 0/797 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/26037 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/797 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/796 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.757400,1.208936,0.606023,0.601160


KeyboardInterrupt: 

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import torch
import pandas as pd
from collections import defaultdict
from datasets import Dataset, DatasetDict, ClassLabel
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)
from sklearn.metrics import accuracy_score, f1_score
import os

# Paths
train_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/combined_all_groups_.csv'
test_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/new_group_.csv'
lexicon_path = '/content/drive/MyDrive/Colab Notebooks/Y2BlC/Transformer model/NRC-Emotion-Lexicon-Wordlevel-v0.92.txt'

# Load dataset
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

# Prepare emotion lexicon with mapping
emotion_mapping = {
    'anger': 'anger',
    'disgust': 'disgust',
    'fear': 'fear',
    'joy': 'happiness',
    'sadness': 'sadness',
    'surprise': 'surprise'
}
emotion_lexicon = defaultdict(list)

with open(lexicon_path, 'r') as f:
    for line in f:
        word, emotion, association = line.strip().split('\t')
        if int(association) == 1 and emotion in emotion_mapping:
            emotion_lexicon[word].append(emotion_mapping[emotion])

# Create emotion count features
def add_lexicon_features(df):
    features = []
    for sentence in df['Translation']:
        counts = defaultdict(int)
        words = str(sentence).lower().split()
        for word in words:
            for emo in emotion_lexicon.get(word, []):
                counts[emo] += 1
        features.append([counts.get(e, 0) for e in sorted(set(emotion_mapping.values()))])
    return pd.DataFrame(features, columns=sorted(set(emotion_mapping.values())))

# Add features to data
train_feats = add_lexicon_features(train)
test_feats = add_lexicon_features(test)
for col in train_feats.columns:
    train[f'lex_{col}'] = train_feats[col]
    test[f'lex_{col}'] = test_feats[col]

# Convert to HuggingFace Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(train),
    "test": Dataset.from_pandas(test)
})

# Split test into validation and test (50/50)
dataset["validation"] = dataset["test"].shard(num_shards=2, index=0)
dataset["test"] = dataset["test"].shard(num_shards=2, index=1)

# Tokenization
model_id = "roberta-large"
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["Translation"], padding="max_length", truncation=True, max_length=256)

dataset = dataset.map(tokenize, batched=True)

# Rename label column in all splits
for split in ["train", "validation", "test"]:
    if "Emotion" in dataset[split].column_names:
        dataset[split] = dataset[split].rename_column("Emotion", "labels")

# Cast label column to ClassLabel
class_names = sorted(train["Emotion"].unique().tolist())
class_label_feature = ClassLabel(names=class_names)
for split in ["train", "validation", "test"]:
    dataset[split] = dataset[split].cast_column("labels", class_label_feature)

# Format datasets for PyTorch
for split in ["train", "validation", "test"]:
    dataset[split].set_format("torch", columns=["input_ids", "attention_mask", "labels"])

# Model config
id2label = {i: label for i, label in enumerate(class_names)}
config = AutoConfig.from_pretrained(model_id, num_labels=len(class_names), id2label=id2label)

# Load model
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

# Training arguments
training_args = TrainingArguments(
    output_dir="/content/results",
    num_train_epochs=8,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=3e-5,
    weight_decay=0.01,
    warmup_steps=500,
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to="tensorboard"
)

# Metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Start training
trainer.train()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Map:   0%|          | 0/26713 [00:00<?, ? examples/s]

Map:   0%|          | 0/458 [00:00<?, ? examples/s]

Map:   0%|          | 0/459 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/26713 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/459 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/458 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.982700,0.682788,0.740741,0.741096
2,0.950000,0.700269,0.749455,0.750019
3,0.622800,0.848347,0.742919,0.742583
4,0.447500,0.989990,0.738562,0.740483


TrainOutput(global_step=6680, training_loss=0.7359238651341307, metrics={'train_runtime': 3447.1048, 'train_samples_per_second': 61.995, 'train_steps_per_second': 3.876, 'total_flos': 4.979020615610573e+16, 'train_loss': 0.7359238651341307, 'epoch': 4.0})